## Cleanly remove all "(USCDI)" tags from short and uscdi extension from StructureDefinitions

1. get each StuctureDefinition in resources-yaml
2. cycle through the elements and look for and remove:
   1.  "(USCDI)" tags in short !! if identical to snapshot then remove the short element altogether
   2.   uscdi extension
3. save each StuctureDefinition in resources-yaml
4. test for errors and issues

In [1]:
##### DO THIS ONCE TO preserve the original files ####
##### DON'T RUN THIS AGAIN ####

# !cp /Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml/Struct*.yml /Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml-old

In [2]:
##### To Start Over ####

# !cp /Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml-old/Struct*.yml /Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml

In [3]:
from yaml import load as yload, dump as ydump, Loader
from json import loads, dumps
from pathlib import Path
from copy import deepcopy
from datetime import datetime

in_path = Path(r'/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml-old')
R4_path = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.r4.core#4.0.1/package')

out_path = Path(r'/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml')


today = datetime.today().strftime('%Y-%m-%d')
today, in_path, out_path

('2023-10-11',
 PosixPath('/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml-old'),
 PosixPath('/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml'))

In [4]:
from yaml import load as yload, dump as ydump, Loader
from json import loads, dumps
from pathlib import Path
from copy import deepcopy
from datetime import datetime

in_path = Path(r'/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml-old')
R4_path = Path(r'/Users/ehaas/.fhir/packages/hl7.fhir.r4.core#4.0.1/package')

out_path = Path(r'/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml')


today = datetime.today().strftime('%Y-%m-%d')
today, in_path, out_path

('2023-10-11',
 PosixPath('/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml-old'),
 PosixPath('/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml'))

In [5]:
def get_shorty(r_type, element_id, shorty):
    for sd_base in R4_path.glob(f'StructureDefinition*{r_type}.json'):
        try:
            r_base = loads(sd_base.read_text())
        except Exception as e:
            print("Exception: {}".format(type(e).__name__))
            print("Exception message: {}".format(e))
        # print(f"======{sd_base}, {r_base['type']}======")
        for element in r_base['differential']['element']:
            if element['id'] == element_id:
                if element['short'] == shorty:
                    return True
                else:
                    return False  



#todo make a deepcopy and modify that copy instead of the original

for sd in in_path.glob('StructureDefinition*.yml'):
    print(f"***********{sd}***********")
    r = yload(sd.read_text(), Loader=Loader)
    r_copy = deepcopy(r)
    print(f"☞ ☞ ☞ Type = {r['type']}")
    for i, element in enumerate(r['differential']['element']):
        print(f"☞ ☞ ☞ element {i}: {element['id']}")

        try:
            print(f"\U00002753 \U00002753 \U00002753 more than 1 extension? = {len(element['extension']) > 1} ❗❗❗")
            for j, extension in enumerate(element['extension']):
                if extension['url'] == 'http://hl7.org/fhir/us/core/StructureDefinition/uscdi-requirement':
                    if len(element['extension']) == 1:

                        print('❗❗❗remove the extension element in the copy')
                        r_copy['differential']['element'][i].pop('extension')
                        r_copy['date'] = today
                    else:
                        print('❗❗❗remove the extension in the copy')
                        r_copy['differential']['element'][i].extension.pop(j)
                        r_copy['date'] = today
                # print(extension['url'], extension['valueBoolean'])
            modified_short = element['short'].replace('(USCDI) ','').replace('(ADD\'L USCDI) ','').replace('(USCDI Requirement) ','') 
            print(f" 👀 modify {element['short']} to {modified_short}")
            if modified_short != element['short']:
                print('❗❗❗modify the short in the copy')
                r_copy['differential']['element'][i]['short'] = modified_short
                r_copy['date'] = today
            # compare short to the R4 fhir element
            shorty = get_shorty(r['type'],element['id'],modified_short)
            print(f"\U00002753 \U00002753 \U00002753 Is the short '{modified_short}' the same as the base - {shorty} ❗❗❗")
            if shorty: 
                print('❗❗❗remove short from the copy')
                r_copy['differential']['element'][i].pop('short')


            print('❗❗❗ done with element')
            print('-'*10)

        except Exception as e:
            print("Exception: {}".format(type(e).__name__))
            print("Exception message: {}".format(e))
            print('-'*10)
    print('❗❗❗ done with SD')
    print('-'*40)
    print('\n')
    # save the copy to out_path using Pathlib
    copyfile = out_path / sd.name
    copyfile.write_text(ydump(r_copy, sort_keys=False, indent=2))


***********/Users/ehaas/Documents/FHIR/USCDI4-Sandbox/input/resources-yaml-old/StructureDefinition-us-core-specimen.yml***********
☞ ☞ ☞ Type = Specimen
☞ ☞ ☞ element 0: Specimen
Exception: KeyError
Exception message: 'extension'
----------
☞ ☞ ☞ element 1: Specimen.identifier
❓ ❓ ❓ more than 1 extension? = False ❗❗❗
❗❗❗remove the extension element in the copy
 👀 modify (USCDI) Specimen identifier to Specimen identifier
❗❗❗modify the short in the copy
❓ ❓ ❓ Is the short 'Specimen identifier' the same as the base - False ❗❗❗
❗❗❗ done with element
----------
☞ ☞ ☞ element 2: Specimen.accessionIdentifier
❓ ❓ ❓ more than 1 extension? = False ❗❗❗
❗❗❗remove the extension element in the copy
 👀 modify (USCDI) Specimen identifier assigned by the lab to Specimen identifier assigned by the lab
❗❗❗modify the short in the copy
❓ ❓ ❓ Is the short 'Specimen identifier assigned by the lab' the same as the base - False ❗❗❗
❗❗❗ done with element
----------
☞ ☞ ☞ element 3: Specimen.type
❓ ❓ ❓ more than